Below is were I am going to pull atleast one file, and begin viewing its contents, extracting ChromaDB Data Entry Points 
that are important to the make-up of the subject involved. The XML files are broken up small enough to view and process on a notebook 
platform. 

In [1]:
import xml.etree.ElementTree as ET
import xml.dom.minidom

xml_file_path = r'/Volumes/NO NAME/xml_chunks/Aalborg Municipality__1.xml'

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()


In [3]:
print(f"Root Element: {root.tag}")
for child in root[:5]:
    print(f"Child element: {child.tag} - {child.attrib}")

Root Element: {http://www.mediawiki.org/xml/export-0.11/}mediawiki
Child element: {http://www.mediawiki.org/xml/export-0.11/}page - {}


In [5]:
# Pretty-print the XML to see the structure
xml_str = ET.tostring(root, encoding='utf-8')
parsed_str = xml.dom.minidom.parseString(xml_str)

# Display the XML in a readable format
 # print(parsed_str.toprettyxml())

In [7]:
# Function to strip namespaces from tags
def strip_namespace(tag):
    return tag.split('}', 1)[-1]  # Remove everything before the '}'

# Set to store unique tag names
tags = set()

# Function to recursively explore all tags in the XML
def extract_tags(element):
    # Add the tag (without namespace) to the tags set
    tags.add(strip_namespace(element.tag))
    
    # Recursively go through each child element
    for child in element:
        extract_tags(child)

# Start extracting tags from the root
extract_tags(root)

# Print out all the unique tags found
print("Unique Tags in the XML:")
for tag in tags:
    print(tag)

Unique Tags in the XML:
timestamp
contributor
minor
id
title
page
mediawiki
revision
model
text
sha1
username
origin
ns
ip
comment
parentid
format


Below I am taking all the data that I want from the xml schema and creating a dataframe to analyze the date inside this 
one xml file. I have 460 xml files to process all together, but once I learn what information I need from one schema, I can change the heading to stream each xml file to get the important information keys from each file. This is just going into one, to find a systematical way to process all of them.

In [23]:

import pandas as pd


# Function to strip namespaces from tags
def strip_namespace(tag):
    return tag.split('}', 1)[-1]  # Remove everything before the '}'

# Initialize an empty list to hold extracted data
data = []

# Iterate through each <page> element
for page in root.findall('.//ns0:page', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}):
    # Extract the title
    title = page.find('.//ns0:title', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}).text if page.find('.//ns0:title', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}) is not None else 'No title'

    # Initialize a list to store revisions for this page
    for revision in page.findall('.//ns0:revision', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}):
        
        # Extract each field within the revision
        revision_data = {
            'title': title,
            'timestamp': revision.find('.//ns0:timestamp', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}).text if revision.find('.//ns0:timestamp', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}) is not None else 'No timestamp',
            'username': revision.find('.//ns0:contributor/ns0:username', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}).text if revision.find('.//ns0:contributor/ns0:username', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}) is not None else 'No username',
            'comment': revision.find('.//ns0:comment', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}).text if revision.find('.//ns0:comment', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}) is not None else 'No comment',
            'text': revision.find('.//ns0:text', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}).text if revision.find('.//ns0:text', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}) is not None else 'No text',
            'sha1': revision.find('.//ns0:sha1', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}).text if revision.find('.//ns0:sha1', namespaces={'ns0': 'http://www.mediawiki.org/xml/export-0.11/'}) is not None else 'No sha1',
            }

        # Add revision data to the list
        data.append(revision_data)

# Convert the list of dictionaries into a Pandas DataFrame
df = pd.DataFrame(data)

# Preview the DataFrame
display(df.head())  # Show first few rows of the DataFrame


title             timestamp           username  \
0  Aalborg Municipality  2002-02-04T03:55:24Z  Conversion script   
1  Aalborg Municipality  2002-02-25T15:43:11Z     imported>31.94   
2  Aalborg Municipality  2002-07-23T03:26:55Z      Magnus Manske   
3  Aalborg Municipality  2002-08-31T05:26:56Z        No username   
4  Aalborg Municipality  2002-09-09T01:24:37Z          The Epopt   

                comment                                               text  \
0  Automated conversion  '''Aalborg''' is a city and seaport of [[Denma...   
1                     *  '''Aalborg''' is a city and seaport of [[Denma...   
2                -/talk  '''Aalborg''' is a city and seaport of [[Denma...   
3             copy-edit  '''Aalborg''' is a city and seaport of [[Denma...   
4     removed self-link  '''Aalborg''' is a city and seaport of [[Denma...   

                              sha1  
0  tkvd4kutgn73kjac7zsi7fz06eri3c5  
1  56wfzuk883j23gvgs35e1v8zwpk1yco  
2  ahmh528xu6po727fb6y64j4nz1ge4dn  
3  75l5zymo21vc9krif06hvp5tttu1i1f  
4  f89w3kj8rfgxzx0xcjfsto0awpterfs

In [11]:
# Show all unique sha1 values
unique_sha1_values = df['sha1'].unique()

print("Unique sha1 values:")
print(unique_sha1_values)

Unique sha1 values:
['tkvd4kutgn73kjac7zsi7fz06eri3c5' '56wfzuk883j23gvgs35e1v8zwpk1yco'
 'ahmh528xu6po727fb6y64j4nz1ge4dn' '75l5zymo21vc9krif06hvp5tttu1i1f'
 'f89w3kj8rfgxzx0xcjfsto0awpterfs' '0aan9isff7c08t160g7mb9qgaytpz0j'
 'syykxq03s0c98jmedhxatweo5vaj8os' 'l042zychv4p7k25gh9gd41qw173xtjt'
 'ef9g3b9yuffr96x1m7ot2dyhqpzbp7k' 'qzt7oze95h72es9qj2nfzrxqd5tvdgh'
 '2dufletsn1h6c2c2lt18yu3352kzyef' 'b6v4z42mm5mkabm1bh71x51vidms1vd'
 '5qc3jntwk3ntxajzu0n1t5z2tzjoeqa' 'No sha1']


In [21]:
sha1_counts = df['sha1'].value_counts()
unique_sha1 = sha1_counts[sha1_counts == 1].index

# Step 3: Filter the DataFrame to show rows with unique sha1 values
unique_sha1_df = df[df['sha1'].isin(unique_sha1)]

# Display the filtered DataFrame
# print(unique_sha1_df)

display(unique_sha1_df)

title             timestamp             username  \
0   Aalborg Municipality  2002-02-04T03:55:24Z    Conversion script   
1   Aalborg Municipality  2002-02-25T15:43:11Z       imported>31.94   
2   Aalborg Municipality  2002-07-23T03:26:55Z        Magnus Manske   
3   Aalborg Municipality  2002-08-31T05:26:56Z          No username   
4   Aalborg Municipality  2002-09-09T01:24:37Z            The Epopt   
5   Aalborg Municipality  2001-06-26T22:58:46Z        KoyaanisQatsi   
6   Aalborg Municipality  2001-09-28T17:58:47Z      213.237.179.xxx   
7   Aalborg Municipality  2002-09-10T23:45:10Z       Christian List   
8   Aalborg Municipality  2002-10-16T16:12:13Z       Christian List   
9   Aalborg Municipality  2002-10-28T07:29:28Z              Olivier   
10  Aalborg Municipality  2002-11-10T09:21:05Z             Jeronimo   
11  Aalborg Municipality  2003-07-14T09:16:51Z  Torkil Bladt~enwiki   
12  Aalborg Municipality  2003-07-14T09:23:44Z  Torkil Bladt~enwiki   
13  Aalborg Municipality  2003-08-28T07:53:03Z         Wazow~enwiki   

                                              comment  \
0                                Automated conversion   
1                                                   *   
2                                              -/talk   
3                                           copy-edit   
4                                   removed self-link   
5   wikifying somewhat and removing 100-year-old f...   
6                                                   *   
7                                            Copyedit   
8                                                 da:   
9                                          No comment   
10                                               edit   
11  Minor corrections, Aalborg University, externa...   
12                                         No comment   
13                                                +pl   

                                                 text  \
0   '''Aalborg''' is a city and seaport of [[Denma...   
1   '''Aalborg''' is a city and seaport of [[Denma...   
2   '''Aalborg''' is a city and seaport of [[Denma...   
3   '''Aalborg''' is a city and seaport of [[Denma...   
4   '''Aalborg''' is a city and seaport of [[Denma...   
5   Aalborg, a city and seaport of [[Denmark]], th...   
6   '''Aalborg''' is a city and seaport of [[Denma...   
7   '''Aalborg''' is a city and seaport of [[Denma...   
8   [[da:Ålborg]]\n'''Aalborg''' is a city and sea...   
9   [[da:Ålborg]]\n'''Aalborg''' is a city and sea...   
10  [[da:Ålborg]]\n'''Aalborg''' ([[Danish languag...   
11  [[da:Ålborg]]\n'''Aalborg''' ([[Danish languag...   
12  [[da:Ålborg]]\n'''Aalborg''' ([[Danish languag...   
13  [[da:Ålborg]][[pl:Aalborg]]\n'''Aalborg''' ([[...   

                               sha1  
0   tkvd4kutgn73kjac7zsi7fz06eri3c5  
1   56wfzuk883j23gvgs35e1v8zwpk1yco  
2   ahmh528xu6po727fb6y64j4nz1ge4dn  
3   75l5zymo21vc9krif06hvp5tttu1i1f  
4   f89w3kj8rfgxzx0xcjfsto0awpterfs  
5   0aan9isff7c08t160g7mb9qgaytpz0j  
6   syykxq03s0c98jmedhxatweo5vaj8os  
7   l042zychv4p7k25gh9gd41qw173xtjt  
8   ef9g3b9yuffr96x1m7ot2dyhqpzbp7k  
9   qzt7oze95h72es9qj2nfzrxqd5tvdgh  
10  2dufletsn1h6c2c2lt18yu3352kzyef  
11  b6v4z42mm5mkabm1bh71x51vidms1vd  
12  5qc3jntwk3ntxajzu0n1t5z2tzjoeqa  
13                          No sha1

Next I want to look inside the text to pull key phrases out that are in [[ __ ]] like da:Aalborg, or Denmark...ect. This builds a hierarchy of relation associated with ontology pyramid. From the text we will create a super and sublevel of classes in columns related to the text.

In [30]:
import re

def extract_unique_links(text):
    # Regular expression to find all substrings in [[ ]]
    links = re.findall(r'\[\[([^\]]+)\]\]', text)
    return set(links)  # Return unique links using a set

# Example: Extract links from the first text in the DataFrame
unique_links_from_first_text = extract_unique_links(unique_sha1_df['text'][0])

# Print the unique links from the first text
print("Unique links from the first text:", unique_links_from_first_text)


Unique links from the first text: {'Store Vildmose', 'Lille Vildmose', 'Budolphi', 'Aalborg', 'talk:Aalborg|Talk', 'Norre Sundby', 'Germany', 'Limfjord', 'North Sea', 'Frederikshavn', 'Skagen', 'Cattegat', 'Aarhus', 'Denmark', 'Jutland', 'Hjorring', 'Frue'}


In [34]:
# text hierarchy
text_df = text_df = unique_sha1_df[['text']]

# Display the 'text' column
display(text_df)


text
0   '''Aalborg''' is a city and seaport of [[Denma...
1   '''Aalborg''' is a city and seaport of [[Denma...
2   '''Aalborg''' is a city and seaport of [[Denma...
3   '''Aalborg''' is a city and seaport of [[Denma...
4   '''Aalborg''' is a city and seaport of [[Denma...
5   Aalborg, a city and seaport of [[Denmark]], th...
6   '''Aalborg''' is a city and seaport of [[Denma...
7   '''Aalborg''' is a city and seaport of [[Denma...
8   [[da:Ålborg]]\n'''Aalborg''' is a city and sea...
9   [[da:Ålborg]]\n'''Aalborg''' is a city and sea...
10  [[da:Ålborg]]\n'''Aalborg''' ([[Danish languag...
11  [[da:Ålborg]]\n'''Aalborg''' ([[Danish languag...
12  [[da:Ålborg]]\n'''Aalborg''' ([[Danish languag...
13  [[da:Ålborg]][[pl:Aalborg]]\n'''Aalborg''' ([[...

In [ ]:
def build_hierarchy(text_df):
    